# `qcrypto` Showcase

This notebook presents the key features of `qcrypto`, a Python library of the simulation of simple quantum cryptography simulations. The primary classes of this library are `QstateEnt` and `QstateUnEnt`. These classes are used to represent the quantum state of a set of qubits. The former is the most general, being capable of simulating the state of a possiibility entangled set of qubits, whereas the latter can only be used to simulate a system of qubits which are unentangled.

In [2]:
from qcrypto.simbasics import QstateEnt, QstateUnEnt

In [22]:
# Entangled set of n qubits
n = 2
qstateent = QstateEnt(init_method="random", init_nbqubits=n)
qstateent

[0.05056588+0.45371015j 0.24549441+0.12567301j 0.7470044 +0.03376755j
 0.17609874+0.35406553j]

In [23]:
qstateent.measure_all("simult")

array([0, 0])

In [ ]:
# Unentangled set of n qubits
n = 2
# qstateunent = QstateUnEnt(init_method="random", )

`qcrypto` also provides methods for applying quantum gates to the qubits, giving it the potential to be used to simulate simple quantum circuits. For instance, we can apply the Hadamard gate:
$$
    H = \frac{1}{\sqrt{2}}\begin{pmatrix}
        1 & 1 \\ 1 & -1
    \end{pmatrix}
$$